In [1]:
import os
import geopandas

In [2]:
pip install pyshp

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\ncbcp\appdata\local\programs\python\python312\lib\site-packages\chatbotai-0.3.1.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install rasterio shapely fiona

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\ncbcp\appdata\local\programs\python\python312\lib\site-packages\chatbotai-0.3.1.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import rasterio
import rasterio.mask
import fiona
import csv
from shapely.geometry import shape, mapping
import numpy as np
from fiona.transform import transform_geom

def reproject_geometry(geometry, src_crs, dst_crs):
    return transform_geom(src_crs, dst_crs, geometry)

# Define the input paths
input_tiff = r"D:/Z/Mahmood/Solar Panel Project/GoogleEarth_image_with_L_Band.tif"
input_shapefile = r"D:/Z/Mahmood/Solar Panel Project/NEW_ROIs_USING ENVI/shapefiles/barren.shp"


# Define the output CSV file path
output_csv = r"D:/Z/Mahmood/Solar Panel Project/NEW_ROIs_USING ENVI/csvs/barren.csv"

# Open the shapefile
with fiona.open(input_shapefile, "r") as shapefile:
    # Read the schema (field names) from the shapefile
    schema = shapefile.schema
    shapefile_crs = shapefile.crs
    
    # Open the hyperspectral TIFF image
    with rasterio.open(input_tiff) as src:
        # Get the raster CRS
        raster_crs = src.crs
        # Get the number of bands in the hyperspectral image
        num_bands = src.count
        
        # Prepare the header for the CSV file
        header = list(schema['properties'].keys()) + ['x', 'y'] + [f'B{i}' for i in range(1, num_bands + 1)]
        
        # Open the CSV file for writing
        with open(output_csv, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            
            # Write the header row
            csvwriter.writerow(header)
            
            # Iterate over each polygon feature in the shapefile
            for feature in shapefile:
                geom = shape(feature['geometry'])
                properties = feature['properties']
                
                # Reproject geometry if CRS are different
                if shapefile_crs != raster_crs:
                    geom = shape(reproject_geometry(mapping(geom), shapefile_crs, raster_crs))
                
                try:
                    # Mask the raster with the current polygon and extract band values
                    masked_image, transform = rasterio.mask.mask(src, [geom], crop=True)
                    
                    # Get the indices of the non-nodata pixels
                    mask = np.ones(masked_image.shape[1:], dtype=bool)
                    if src.nodata is not None:
                        mask = (masked_image != src.nodata).all(axis=0)
                    
                    # Get the coordinates of the non-nodata pixels
                    row_indices, col_indices = np.where(mask)
                    for row, col in zip(row_indices, col_indices):
                        pixel_values = masked_image[:, row, col]
                        
                        # Prepare a list to hold the row data
                        row_data = list(properties.values())
                        
                        # Get the pixel coordinates in the original image space
                        x, y = rasterio.transform.xy(transform, row, col)
                        
                        # Append coordinates and pixel values to the row
                        row_data.extend([x, y])
                        row_data.extend(pixel_values)
                        
                        # Write the row to the CSV file
                        csvwriter.writerow(row_data)
                except ValueError as e:
                    print(f"Skipping feature {feature['id']}: {e}")

print(f"CSV file with hyperspectral data has been created: {output_csv}")


CSV file with hyperspectral data has been created: D:/Z/Mahmood/Solar Panel Project/NEW_ROIs_USING ENVI/csvs/barren.csv
